In [282]:
import nltk
import string
import pandas as pd
import copy

<h1>Reading training data from a file</h1>

In [285]:
data=pd.read_csv('medium_articles.csv')

<br><h3><i>Removing unnecessary data from the text dataset</i></h3><br>

In [287]:
data.drop('title', axis=1, inplace=True)
data.drop('authors', axis=1, inplace=True)
data.drop('timestamp', axis=1, inplace=True)
data.drop('url', axis=1, inplace=True)

data = data.dropna(subset = ['text'])
data = data.iloc[:100]
data[:6]



,text,tags
0,Photo by Josh Riemer on Unsplash\n\nMerry Chri...,"['Mental Health', 'Health', 'Psychology', 'Sci..."
1,Your Brain On Coronavirus\n\nA guide to the cu...,"['Mental Health', 'Coronavirus', 'Science', 'P..."
2,Mind Your Nose\n\nHow smell training can chang...,"['Biotechnology', 'Neuroscience', 'Brain', 'We..."
3,Passionate about the synergy between science a...,"['Health', 'Neuroscience', 'Mental Health', 'P..."
4,"You’ve heard of him, haven’t you? Phineas Gage...","['Brain', 'Health', 'Development', 'Psychology..."
5,"Mentally, Young Adults Are Suffering Most From...","['Society', 'Mental Health', 'Health', 'Nonfic..."


<br><h3><i>Removing punctuation</i></h3><br>

In [289]:
punctuation = string.punctuation
def remove_punc():
    data['text'] = data['text'].apply(lambda x: x.lower())
    for index in range(0, len(data)):
        for s in punctuation:
            data.loc[index, 'text'] = data.loc[index, 'text'].replace(s, ' ')
        data.loc[index, 'text'] = data.loc[index, 'text'].replace('\n', ' ').replace('“', '').replace('”','').replace('—','').replace("’","`")
remove_punc()

In [338]:
'us' in stop_words

False

<br><h3><i>Spliting text</i></h3><br>

In [292]:
data[0:5]

,text,tags
0,photo by josh riemer on unsplash merry christ...,"['Mental Health', 'Health', 'Psychology', 'Sci..."
1,your brain on coronavirus a guide to the curi...,"['Mental Health', 'Coronavirus', 'Science', 'P..."
2,mind your nose how smell training can change ...,"['Biotechnology', 'Neuroscience', 'Brain', 'We..."
3,passionate about the synergy between science a...,"['Health', 'Neuroscience', 'Mental Health', 'P..."
4,you`ve heard of him haven`t you phineas gage...,"['Brain', 'Health', 'Development', 'Psychology..."


In [293]:
def split(data):
    data['vectors'] = data['text'].apply(lambda row: row.split(' '))
    for i in range (0,len(data)):
        vectors = data.loc[i, 'vectors']
        for word in vectors:
            if not word.strip():
                vectors.remove(word)
        data.at[i, 'vectors'] = vectors

split(data)
print(data[0:4]['vectors'])

0    [photo, by, josh, riemer, on, unsplash, merry,...
1    [your, brain, on, coronavirus, a, guide, to, t...
2    [mind, your, nose, how, smell, training, can, ...
3    [passionate, about, the, synergy, between, sci...
Name: vectors, dtype: object


<br><h3><i>Removing stop words</i></h3><br>

In [295]:
stop_words=nltk.corpus.stopwords.words('english')
stop_words = [word.replace("'", "`") for word in stop_words]
data['count'] = data['vectors'].apply(lambda x: len(x))

In [296]:
'that`s' in stop_words

False

In [297]:
def remove_stopwords():
    for i in range (0, len(data)):
        words = data.loc[i, 'vectors']
        removed_words = 0
        length = len(words)
        for index in range(0, length):
            if words[index - removed_words] in stop_words:
                words.remove(words[index - removed_words])
                removed_words = removed_words + 1
        data.at[i, 'vectors'] = words
remove_stopwords()
data['count'] = data['vectors'].apply(lambda x: len(x))

<br><h3><i>Stemming</i></h3><br>

In [315]:
data[0:1]

,text,tags,vectors,count,stemmed
0,photo by josh riemer on unsplash merry christ...,"['Mental Health', 'Health', 'Psychology', 'Sci...","[photo, josh, riemer, unsplash, merry, christm...",450,"[photo, josh, riemer, unsplash, merri, christm..."


In [323]:
def stem_text():
    ps = nltk.PorterStemmer() 
    data['stemmed'] = data['vectors'].apply(lambda x: copy.deepcopy(x))
    for index in range(0, len(data)):
        words = data.loc[index, 'stemmed']
        for i in range(0, len(words)):
            words[i] = ps.stem(words[i])
        data.at[index, 'stemmed'] = words
stem_text()
print(data[['stemmed', 'vectors']])

                                              stemmed  \
0   [photo, josh, riemer, unsplash, merri, christm...   
1   [brain, coronaviru, guid, curiou, troubl, impa...   
2   [mind, nose, smell, train, chang, brain, six, ...   
3   [passion, synergi, scienc, technolog, provid, ...   
4   [heard, phinea, gage, railroad, worker, surviv...   
..                                                ...   
95  [solitud, stigma, societi, harshli, judg, solo...   
96  [think, work, googl, like, come, mind, legenda...   
97  [j, k, rowling`, advic, writer, big, dream, si...   
98  [septemb, 22, mark, begin, autumn, remind, won...   
99  [mesensei, compani, design, creat, mobil, solu...   

                                              vectors  
0   [photo, josh, riemer, unsplash, merry, christm...  
1   [brain, coronavirus, guide, curious, troubling...  
2   [mind, nose, smell, training, change, brain, s...  
3   [passionate, synergy, science, technology, pro...  
4   [heard, phineas, gage, railroad

<br><h3><i>Lemmatizing</i></h3><br>

In [336]:
def lemmatize_text():
    lm = nltk.WordNetLemmatizer() 
    data['lammatized'] = data['vectors'].apply(lambda x: copy.deepcopy(x))
    for index in range(0, len(data)):
        words = data.loc[index, 'lammatized']
        for i in range(0, len(words)):
            words[i] = lm.lemmatize(words[i])
        data.at[index, 'lammatized'] = words
lemmatize_text()

Start: sens, end: sen
Start: sens, end: sen
Start: sens, end: sen
Start: pas, end: pa
Start: pas, end: pa
Start: pass, end: pas
Start: pass, end: pas
Start: cs, end: c
Start: us, end: u
Start: pas, end: pa
Start: pass, end: pas
Start: us, end: u
Start: pas, end: pa
Start: us, end: u
Start: sens, end: sen
Start: us, end: u
Start: pas, end: pa
Start: as, end: a
Start: us, end: u
Start: us, end: u


<br><h1><strong>Vectorizing data</strong></h1><br>

<br><h3><i>Count vectorising</i></h3><br>

In [351]:
from sklearn.feature_extraction.text import CountVectorizer

In [373]:
count_vect = CountVectorizer()
X_counts_stemmed = count_vect.fit_transform(data['stemmed'].apply(lambda x: ' '.join(x)))
# X_counts_lemmatized = count_vect.fit_transform(data['lammatized'].apply(lambda x: ' '.join(x)))

In [383]:
print(X_counts_stemmed.shape)

(100, 6515)


In [389]:
print(count_vect.get_feature_names_out()[345])

albanesiu


<br><h3><i>N gramm vectorising</i></h3><br>

In [409]:
n_count_vect = CountVectorizer(ngram_range = (2,3))
ngram_lemmatized = n_count_vect.fit_transform(data['lammatized'].apply(lambda x: ' '.join(x)))

In [411]:
print(ngram_lemmatized.shape)
print(n_count_vect.get_feature_names_out())

(100, 83266)
['000 american' '000 american 700' '000 common' ... '⁸state ai ethic'
 '⁹insurance product' '⁹insurance product ¹⁰data']


In [415]:
pd.DataFrame(ngram_lemmatized.toarray())

,0,1,2,3,4,5,6,7,8,9,...,83256,83257,83258,83259,83260,83261,83262,83263,83264,83265
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
97,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<br><h3><i>Frequency weighting</i></h3><br>

In [419]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [431]:
tfidf_vect_stemmed = TfidfVectorizer()
X_tfidf_stem = tfidf_vect_stemmed.fit_transform(data['stemmed'].apply(lambda x: ' '.join(x)))

In [435]:
print(X_tfidf_stem.shape)
print(tfidf_vect_stemmed.get_feature_names_out())

(100, 6515)
['000' '00041' '025' ... '⁷ai' '⁸state' '⁹insur']


In [441]:
pd.DataFrame(X_tfidf_stem.toarray())

,0,1,2,3,4,5,6,7,8,9,...,6505,6506,6507,6508,6509,6510,6511,6512,6513,6514
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.028749,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,0.017571,0.0,0.0,0.0,0.0,0.0,0.0,0.013310,0.01931,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
